#### Result evaluation

In [9]:
import numpy as np
import cv2
import os

""" 
Confusion matrix
P\L     P    N 
P      TP    FP 
N      FN    TN 
"""  

# same def in Train_model
def color_dict(labelFolder, classNum):
    colorDict = []
    ImageNameList = os.listdir(labelFolder)
    for i in range(len(ImageNameList)):
        ImagePath = labelFolder + "/" + ImageNameList[i]
        img = cv2.imread(ImagePath).astype(np.uint32)
        if(len(img.shape) == 2):
            img = cv2.cvtColor(img, cv2.COLOR_GRAY2RGB).astype(np.uint32)
        img_new = img[:,:,0] * 1000000 + img[:,:,1] * 1000 + img[:,:,2]
        unique = np.unique(img_new)
        for j in range(unique.shape[0]):
            colorDict.append(unique[j])
        colorDict = sorted(set(colorDict))
        if(len(colorDict) == classNum):
            break
    colorDict_BGR = []
    for k in range(len(colorDict)):
        color = str(colorDict[k]).rjust(9, '0')
        color_BGR = [int(color[0 : 3]), int(color[3 : 6]), int(color[6 : 9])]
        colorDict_BGR.append(color_BGR)
    colorDict_BGR = np.array(colorDict_BGR)
    colorDict_GRAY = colorDict_BGR.reshape((colorDict_BGR.shape[0], 1 ,colorDict_BGR.shape[1])).astype(np.uint8)
    colorDict_GRAY = cv2.cvtColor(colorDict_GRAY, cv2.COLOR_BGR2GRAY)
    return colorDict_BGR, colorDict_GRAY

def ConfusionMatrix(numClass, imgPredict, Label):  
    #  Return confusion matrix
    mask = (Label >= 0) & (Label < numClass)  
    label = numClass * Label[mask] + imgPredict[mask]  
    count = np.bincount(label, minlength = numClass**2)  
    confusionMatrix = count.reshape(numClass, numClass)  
    return confusionMatrix

def OverallAccuracy(confusionMatrix):  
    #  Return overall accuracy
    # acc = (TP + TN) / (TP + TN + FP + TN)  
    OA = np.diag(confusionMatrix).sum() / confusionMatrix.sum()  
    return OA
  
def Precision(confusionMatrix):  
    #  Return precision for each class
    precision = np.diag(confusionMatrix) / confusionMatrix.sum(axis = 0)
    return precision  

def Recall(confusionMatrix):
    #  Return recall for each class
    recall = np.diag(confusionMatrix) / confusionMatrix.sum(axis = 1)
    return recall
  
def F1Score(confusionMatrix):
    precision = np.diag(confusionMatrix) / confusionMatrix.sum(axis = 0)
    recall = np.diag(confusionMatrix) / confusionMatrix.sum(axis = 1)
    f1score = 2 * precision * recall / (precision + recall)
    return f1score
def IntersectionOverUnion(confusionMatrix):  
    #  Return IoU
    intersection = np.diag(confusionMatrix)  
    union = np.sum(confusionMatrix, axis = 1) + np.sum(confusionMatrix, axis = 0) - np.diag(confusionMatrix)  
    IoU = intersection / union
    return IoU

def MeanIntersectionOverUnion(confusionMatrix):  
    #  Return mIoU
    intersection = np.diag(confusionMatrix)  
    union = np.sum(confusionMatrix, axis = 1) + np.sum(confusionMatrix, axis = 0) - np.diag(confusionMatrix)  
    IoU = intersection / union
    mIoU = np.nanmean(IoU)  
    return mIoU
  
def Frequency_Weighted_Intersection_over_Union(confusionMatrix):
    #  Return FWIoU
    freq = np.sum(confusionMatrix, axis=1) / np.sum(confusionMatrix)  
    iu = np.diag(confusionMatrix) / (
            np.sum(confusionMatrix, axis = 1) +
            np.sum(confusionMatrix, axis = 0) -
            np.diag(confusionMatrix))
    FWIoU = (freq[freq > 0] * iu[freq > 0]).sum()
    return FWIoU



#  Predict image PATH
PredictPath = r"evaluation\predict"

#  Predict label PATH
LabelPath = r"evaluation\label"
#  Number of class
classNum = 5

#  Get category color dictionary
colorDict_BGR, colorDict_GRAY = color_dict(LabelPath, classNum)

#  Read all the images in the folder
labelList = os.listdir(LabelPath)
PredictList = os.listdir(PredictPath)

#  To read the shape of the image
Label0 = cv2.imread(LabelPath + "//" + labelList[0], 0)

#  Number of images
label_num = len(labelList)

#  Put all images in an array
label_all = np.zeros((label_num, ) + Label0.shape, np.uint8)
predict_all = np.zeros((label_num, ) + Label0.shape, np.uint8)
for i in range(label_num):
    Label = cv2.imread(LabelPath + "//" + labelList[i])
    Label = cv2.cvtColor(Label, cv2.COLOR_BGR2GRAY)
    label_all[i] = Label
    Predict = cv2.imread(PredictPath + "//" + PredictList[i])
    Predict = cv2.cvtColor(Predict, cv2.COLOR_BGR2GRAY)
    predict_all[i] = Predict

for i in range(colorDict_GRAY.shape[0]):
    label_all[label_all == colorDict_GRAY[i][0]] = i
    predict_all[predict_all == colorDict_GRAY[i][0]] = i

#  flatten label
label_all = label_all.flatten()
predict_all = predict_all.flatten()

#  Calculate confusion matrix and various precision parameters
confusionMatrix = ConfusionMatrix(classNum, predict_all, label_all)
precision = Precision(confusionMatrix)
recall = Recall(confusionMatrix)
OA = OverallAccuracy(confusionMatrix)
IoU = IntersectionOverUnion(confusionMatrix)
FWIOU = Frequency_Weighted_Intersection_over_Union(confusionMatrix)
mIOU = MeanIntersectionOverUnion(confusionMatrix)
f1ccore = F1Score(confusionMatrix)


print("植被      建筑      裸土      水体      背景")
print("Vegetation  building  soil  water  background")
print("ConfusionMatrix:")
print(confusionMatrix)
print("Precision:")
print(precision)
print("Recall:")
print(recall)
print("F1-Score:")
print(f1ccore)
print("OverallAccuracy:")
print(OA)
print("IoU:")
print(IoU)
print("mIoU:")
print(mIOU)
print("FWIoU:")
print(FWIOU)

植被      建筑      裸土      水体      背景
Vegetation  building  soil  water  background
ConfusionMatrix:
[[1179532    1407       0    1625   44805]
 [    157   37725       0       0   19925]
 [    123   10556       0       3    2887]
 [   2323    1313       0  111430   14793]
 [ 103732   22175       0    4705  690784]]
Precision:
[0.91730482 0.51553788        nan 0.9462225  0.89341614]
Recall:
[0.96102476 0.65260263 0.         0.85808454 0.84098778]
F1-Score:
[0.93865598 0.57602895        nan 0.90000081 0.86640955]
OverallAccuracy:
0.8975426666666667
IoU:
[0.88440314 0.40452294 0.         0.81818315 0.76430561]
mIoU:
0.5742829679503395
FWIoU:
0.8190752312638379


<ipython-input-9-f1a0b992dd0d>:66: RuntimeWarning: invalid value encountered in true_divide
  precision = np.diag(confusionMatrix) / confusionMatrix.sum(axis = 0)
<ipython-input-9-f1a0b992dd0d>:75: RuntimeWarning: invalid value encountered in true_divide
  precision = np.diag(confusionMatrix) / confusionMatrix.sum(axis = 0)
